## Task ba6d
based a bit on this video and its later - https://www.youtube.com/watch?v=naovHyR0eEg

In [1]:
from collections import defaultdict

In [51]:
def read_data(fname='rosalind_ba6d.txt'):
    with open(fname, 'r') as f:
        p = [int(elem) for elem in (f.readline().rstrip()[1:-1].split())]
        q = [int(elem) for elem in (f.readline().rstrip()[1:-1].split())]
    return p, q

In [52]:
def get_adj_mtx(p):
    # make a graph out of circular chromosome - treat each element as directed edge (plus and minus sides)
    # edges between elements are undirected
    graph = defaultdict(list)
    for i, elem in enumerate(p[:-1]):
        next_elem = p[i+1]
        graph[elem].append(-next_elem)
        graph[-next_elem].append(elem)
    # connect the circle
    graph[p[-1]].append(-p[0])
    graph[-p[0]].append(p[-1])
    return graph

In [53]:
def get_breakpoint_graph(g_p, g_q):
    # since synteny blocks are the same we just combine (superimpose) edges
    g_br = defaultdict(list)
    for elem in g_p:
        g_br[elem] = g_p[elem] + g_q[elem]
    return g_br

In [134]:
def get_connected_component(g_p, elem, chromosome=True):
    q = [elem]
    visited = list()
    while q:
        cur_elem = q.pop(0)
        visited.append(cur_elem)
        for n in g_p[cur_elem]:
            if chromosome:
                if -n not in visited:
                    q.append(-n)
            else:
                if n not in visited:
                    q.append(n)
    return visited if chromosome else list(set(visited))

In [136]:
def get_all_connected_components(g):
    elems = set(g)
    comps = []
    while elems:
        cur_elem = next(iter(elems))
        res = list(set(get_connected_component(g, cur_elem, chromosome=False)))
        comps.append(res)
        elems -= set(res)
    return comps

In [137]:
def get_reconstructed_chromosome(g_p):
    
    def output_format(lst):
        return "(" + " ".join([str(x) if x < 0 else '+' + str(x) for x in lst]) + ")"

    elems = set(g_p)
    cycles = []
    while elems:
        cmp = get_connected_component(g_p, next(iter(elems)))
        elems -= set(cmp) | set(-x for x in cmp)
        cycles.append(cmp)
    return "".join([output_format(c) for c in cycles])

In [138]:
def get_nodes_for_big_componens(g):
    # if component is only size 2 (smallest cycle), then we ignore these elements
    comps = get_all_connected_components(g)
    res = []
    for c in comps:
        if len(c) > 2:
            return c
    return []

In [139]:
def add_edge(g, i, j):
    g[i].append(j)
    g[j].append(i)


def delete_edge(g, i, j):
    g[i].remove(j)
    g[j].remove(i)

In [148]:
def apply_2_break(g_p, g_q):
    g_br = get_breakpoint_graph(g_p, g_q)
    elems = get_nodes_for_big_componens(g_br)
    print(get_reconstructed_chromosome(g_p))

    while 1:
        if len(elems) == 0:
            break

        # red edge 1
        i = g_p[elems[0]][0]
        j = elems[0]
        # red edge 2
        k = g_q[elems[0]][0]
        l = g_p[k][0]
#         print('i, j, k, l')
#         print(i, j, k, l)
        # delete red edges
        delete_edge(g_p, i, j)
        delete_edge(g_p, k, l)
        
        # add red edges
        add_edge(g_p, j, k)
        add_edge(g_p, l, i)
        
        print(get_reconstructed_chromosome(g_p))
        
        g_br = get_breakpoint_graph(g_p, g_q)
        elems = get_nodes_for_big_componens(g_br)


In [149]:
def main(fname='rosalind_ba6d.txt'):
    p, q = read_data(fname)
    g_p = get_adj_mtx(p)
    g_q = get_adj_mtx(q)
    apply_2_break(g_p, g_q)

In [155]:
main('sample_data/ba6d/sample.txt')

(+1 -2 -3 +4)
(+1 +2 -3 +4)
(+1 +2 -4 +3)
(+1 +2 -4 -3)


In [158]:
# driver code
# main('rosalind_ba6d.txt')